# {mod}`ultralytics` 预测模式

预测模式：释放模型对真实世界数据的预测能力。

In [1]:
import set_env

```{admonition} 预测模式的主要功能
- 兼容多种数据源：无论您的数据是单个图像、图像集合、视频文件还是实时视频流，预测模式都能满足您的需求。
- 流媒体模式： 使用流功能生成具有内存效率的 `Results` 对象。通过设置 `stream=True` 在预测器的调用方法中。
- 批处理：批量处理：能够一次性处理多个图像或视频帧，进一步加快推理时间。
- 易于集成：凭借灵活的应用程序接口，可轻松与现有数据管道和其他软件组件集成。
```

Ultralytics YOLO 模型返回 Python `Results` 对象，或内存效率高的 Python `Results` 当 `stream=True` 会在推理过程中传递给模型：

:::::{tab-set}
::::{tab-item} 返回带有 `stream=False`
```python
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model(["image1.jpg", "image2.jpg"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk
```
::::

::::{tab-item} 返回带有 `stream=True`
```python
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model(["image1.jpg", "image2.jpg"], stream=True)  # return a generator of Results objects

# Process results generator
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk
```
::::
:::::

## 推理源

YOLOv11 可以处理不同种类的输入源进行推理，如下表所示。这些来源包括静态图像、视频流和各种数据格式。表格还指出了每种来源是否可以在带有 `stream=True` 参数的流模式下使用 ✅。流模式对于处理视频或直播非常有用，因为它会创建一个结果生成器，而不是将所有帧加载到内存中。

```{tip}
在处理长视频或大型数据集时，使用`stream=True`可以高效地管理内存。当`stream=False`时，所有帧或数据点的结果都存储在内存中，这可能会导致内存快速增加，并在输入较大时引发内存不足错误。相比之下，`stream=True`采用生成器方式，仅保留当前帧或数据点的结果在内存中，显著降低了内存消耗，并避免了内存不足的问题。
```

| 数据源                                                 | 示例                                   | 类型              | 备注                                                                                           |
| ----------------------------------------------------- | ------------------------------------------ | ---------------- | ------------------------------------------------------------------------------------------- |
| 图像                                                  | `'image.jpg'`                            | `str` 或 `Path`   | 单个图像文件。                                                                                  |
| URL                                                  | `'https://ultralytics.com/images/bus.jpg'` | `str`            | 图像的URL。                                                                                     |
| 屏幕截图                                              | `'screen'`                               | `str`            | 截取屏幕截图。                                                                                  |
| PIL                                                  | `Image.open('image.jpg')`                 | `PIL.Image`      | HWC格式，具有RGB通道。                                                                          |
| [OpenCV](https://www.ultralytics.com/glossary/opencv)  | `cv2.imread('image.jpg')`                 | `np.ndarray`     | HWC格式，具有BGR通道，像素值为`uint8 (0-255)`。                                               |
| numpy                                                | `np.zeros((640,1280,3))`                  | `np.ndarray`     | HWC格式，具有BGR通道，像素值为`uint8 (0-255)`。                                               |
| torch                                                | `torch.zeros(16,3,320,640)`               | `torch.Tensor`   | BCHW格式，具有RGB通道，像素值为`float32 (0.0-1.0)`。                                            |
| CSV                                                  | `'sources.csv'`                           | `str` 或 `Path`   | 包含图像、视频或目录路径的CSV文件。                                                               |
| 视频 &#9989;                                               | `'video.mp4'`                            | `str` 或 `Path`   | 格式为MP4、AVI等的视频文件。                                                                   |
| 目录 &#9989;                                               | `'path/'`                                 | `str` 或 `Path`   | 包含图像或视频的目录路径。                                                                      |
| Glob模式 &#9989;                                           | `'path/*.jpg'`                            | `str`            | 用于匹配多个文件的Glob模式。使用`*`字符作为通配符。                                             |
| YouTube &#9989;                                            | `'https://youtu.be/LNwODJXcvt4'`          | `str`            | YouTube视频的URL。                                                                             |
| 流媒体 &#9989;                                             | `'rtsp://example.com/media.mp4'`          | `str`            | 流媒体协议的URL，例如RTSP、RTMP、TCP或IP地址。                                                   |
| 多路流媒体 &#9989;                                         | `'list.streams'`                          | `str` 或 `Path`   | `*.streams`文本文件，每行一个流媒体URL，即8个流媒体将以batch-size 8运行。                          |
| Webcam &#9989;                                             | `0`                                       | `int`            | 要进行推理的连接摄像头设备的索引。                                                               |

下面是使用每种源类型的代码示例：

:::::{tab-set}
::::{tab-item} 图像
在图像上推理
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define path to the image file
source = "path/to/image.jpg"

# Run inference on the source
results = model(source)  # list of Results objects
```
::::

::::{tab-item} screenshot
将当前屏幕内容作为截图进行推理。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define current screenshot as source
source = "screen"

# Run inference on the source
results = model(source)  # list of Results objects
```
::::

::::{tab-item} URL
通过URL远程托管的图像或视频上运行推理。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define remote image or video URL
source = "https://ultralytics.com/images/bus.jpg"

# Run inference on the source
results = model(source)  # list of Results objects
```
::::


::::{tab-item} PIL
使用Python图像库（PIL）打开的图像上运行推断。
```python
from PIL import Image
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Open an image using PIL
source = Image.open("path/to/image.jpg")

# Run inference on the source
results = model(source)  # list of Results objects
```
::::

::::{tab-item} OpenCV
使用 OpenCV 打开的图像上运行推断。
```python
import cv2
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Read an image using OpenCV
source = cv2.imread("path/to/image.jpg")

# Run inference on the source
results = model(source)  # list of Results objects
```
::::

::::{tab-item} numpy
对以numpy数组表示的图像进行推理。
```python
import numpy as np
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Create a random numpy array of HWC shape (640, 640, 3) with values in range [0, 255] and type uint8
source = np.random.randint(low=0, high=255, size=(640, 640, 3), dtype="uint8")

# Run inference on the source
results = model(source)  # list of Results objects
```
::::

::::{tab-item} numpy
对以 [PyTorch](https://www.ultralytics.com/glossary/pytorch) 张量表示的图像进行推理。
```python
import torch
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Create a random torch tensor of BCHW shape (1, 3, 640, 640) with values in range [0, 1] and type float32
source = torch.rand(1, 3, 640, 640, dtype=torch.float32)

# Run inference on the source
results = model(source)  # list of Results objects
```
::::

::::{tab-item} CSV
在CSV文件中列出的一系列图像、URL、视频和目录上运行推理。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define a path to a CSV file with images, URLs, videos and directories
source = "path/to/file.csv"

# Run inference on the source
results = model(source)  # list of Results objects
```
::::

::::{tab-item} 视频
对视频文件进行推理。通过使用 `stream=True`，你可以创建生成器来生成结果对象，从而减少内存使用量。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define path to video file
source = "path/to/video.mp4"

# Run inference on the source
results = model(source, stream=True)  # generator of Results objects
```
::::

::::{tab-item} 文件夹
对目录中的所有图像和视频进行推理。要捕获子目录中的图像和视频，请使用通配符模式，例如 `path/to/dir/**/*`。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define path to directory containing images and videos for inference
source = "path/to/dir"

# Run inference on the source
results = model(source, stream=True)  # generator of Results objects
```
::::

::::{tab-item} 通配符
对符合带有 `*` 字符的通配符(glob)表达式的所有图像和视频执行推理。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define a glob search for all JPG files in a directory
source = "path/to/dir/*.jpg"

# OR define a recursive glob search for all JPG files including subdirectories
source = "path/to/dir/**/*.jpg"

# Run inference on the source
results = model(source, stream=True)  # generator of Results objects
```
::::


::::{tab-item} YouTube
在YouTube视频上运行推理。通过使用 `stream=True`，你可以创建 Results 对象的生成器来减少长视频的内存占用。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Define source as YouTube video URL
source = "https://youtu.be/LNwODJXcvt4"

# Run inference on the source
results = model(source, stream=True)  # generator of Results objects
```
::::

::::{tab-item} Stream
使用流模式通过RTSP、RTMP、TCP或IP地址协议对实时视频流进行推理。如果只提供一个流，模型将以1的批处理大小运行推理。对于多个流，可以使用`.streams`文本文件执行批量推理，其中批处理大小由提供的流的数量决定（例如，8个流对应批处理大小为8）。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Single stream with batch-size 1 inference
source = "rtsp://example.com/media.mp4"  # RTSP, RTMP, TCP, or IP streaming address

# Run inference on the source
results = model(source, stream=True)  # generator of Results objects
```
对于单流使用，批处理大小默认设置为`1`，这样可以高效地实时处理视频流。
::::

::::{tab-item} Multi-Stream
要同时处理多个视频流，请使用包含流媒体源的 `.streams` 文本文件。模型将运行批处理推理，其中批处理大小等于流的数量。这种设置能够高效地并行处理多个输入。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Multiple streams with batched inference (e.g., batch-size 8 for 8 streams)
source = "path/to/list.streams"  # *.streams text file with one streaming address per line

# Run inference on the source
results = model(source, stream=True)  # generator of Results objects
```

`.streams` 文本文件例子
```
rtsp://example.com/media1.mp4
rtsp://example.com/media2.mp4
rtmp://example2.com/live
tcp://192.168.1.100:554
```
文件中的每一行代表一个流媒体源，使得你能够同时监控并对多个视频流进行推断。
::::

::::{tab-item} Webcam
您可以通过对特定摄像头的索引传递给 `source`，在连接的相机设备上运行推断。
```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Run inference on the source
results = model(source=0, stream=True)  # generator of Results objects
```
::::
:::::

## 推理参数

`model.predict()` 函数在推理时接受多个参数，这些参数可以用来覆盖默认设置：

```python
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Run inference on 'bus.jpg' with arguments
model.predict("bus.jpg", save=True, imgsz=320, conf=0.5)
```

以下是翻译后的内容：

| 参数           | 类型           | 默认值                 | 描述                                                                                                                                                                                                                                                                                                                    |
| -------------- | -------------- | ---------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `source`       | `str`          | `'ultralytics/assets'` | 指定推理的数据源。可以是图像路径、视频文件、目录、URL或实时流的设备ID。支持多种格式和来源，使应用在不同输入类型上具有灵活性。                                                                                                                                                               |
| `conf`         | `float`        | `0.25`                 | 设置检测的最小置信度阈值。低于此阈值的检测结果将被忽略。调整此值有助于减少误报。                                                                                                                                                                                                              |
| `iou`          | `float`        | `0.7`                  | [交并比](https://www.ultralytics.com/glossary/intersection-over-union-iou) (IoU) 非极大值抑制 (NMS) 的阈值。较低的值通过消除重叠框来减少检测数量，有助于减少重复项。                                                                                                                                     |
| `imgsz`        | `int or tuple` | `640`                  | 定义推理的图像大小。可以是一个整数 `640` 用于方形调整，也可以是一个（高度, 宽度）元组。适当的尺寸可以提高检测的准确性和处理速度。                                                                                                                                                             |
| `half`         | `bool`         | `False`                | 启用半精度（FP16）推理，可以在支持的GPU上加速模型推理，对准确性的影响很小。                                                                                                                                                                                                                   |
| `device`       | `str`          | `None`                 | 指定推理的设备（例如，`cpu`、`cuda:0` 或 `0`）。允许用户在CPU、特定GPU或其他计算设备之间选择执行模型。                                                                                                                                                                                           |
| `max_det`      | `int`          | `300`                  | 每张图像允许的最大检测数量。限制模型在单次推理中可以检测到的对象总数，防止在密集场景中产生过多输出。                                                                                                                                                                                                |
| `vid_stride`   | `int`          | `1`                    | 视频输入的帧步幅。允许跳过视频中的帧以加快处理速度，但会牺牲时间分辨率。值为1时处理每一帧，更高值则跳过帧。                                                                                                                                                                                  |
| `stream_buffer`| `bool`         | `False`                | 确定是否为视频流排队传入帧。如果为 `False`，旧帧会被丢弃以适应新帧（优化实时应用）。如果为 `True`，则会在新缓冲区中排队新帧，确保不会跳过任何帧，但如果推理FPS低于流FPS，会导致延迟。                                                                                   |
| `visualize`    | `bool`         | `False`                | 激活推理期间模型特征的可视化，提供关于模型“看到”内容的洞察。对于调试和模型解释非常有用。                                                                                                                                                                                                     |
| `augment`      | `bool`         | `False`                | 启用测试时数据增强（TTA），可能提高检测的鲁棒性，但会牺牲推理速度。                                                                                                                                                                                                                           |
| `agnostic_nms` | `bool`         | `False`                | 启用类无关的非极大值抑制（NMS），合并不同类的重叠框。在多类检测场景中非常有用，其中类重叠是常见的情况。                                                                                                                                                                                              |
| `classes`      | `list[int]`    | `None`                 | 过滤预测结果为一组类ID。只有属于指定类的检测结果才会返回。在多类检测任务中聚焦于相关对象时非常有用。                                                                                                                                                                                                |
| `retina_masks` | `bool`         | `False`                | 如果模型中可用，使用高分辨率分割掩码。这可以增强分割任务的掩码质量，提供更精细的细节。                                                                                                                                                                                                          |
| `embed`        | `list[int]`    | `None`                 | 指定从中提取特征向量或[嵌入](https://www.ultralytics.com/glossary/embeddings)的层。对于下游任务如聚类或相似性搜索非常有用。                                                                                                                                                                                                 |
| `project`      | `str`          | `None`                 | 保存预测输出的项目目录名称。如果启用了 `save`，将在项目文件夹中创建一个子目录，存储预测输出。                                                                                                                                                                                                      |
| `name`         | `str`          | `None`                 | 预测运行的名称。用于在项目文件夹中创建子目录，如果启用了 `save`，将在其中存储预测输出。                                                                                                                                                                                                       |

可视化参数

| 参数          | 类型           | 默认值         | 描述                                                                                                                                                                            |
| ------------- | -------------- | --------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `show`        | `bool`         | `False`         | 如果为 `True`，则在窗口中显示带有标注的图片或视频。在开发或测试期间提供即时视觉反馈非常有用。                                                       |
| `save`        | `bool`         | `False` 或 `True` | 启用将带标注的图片或视频保存到文件。对于文档记录、进一步分析或分享结果非常有用。使用 CLI 时默认值为 True，而在 Python 中使用则默认值为 False。                |
| `save_frames` | `bool`         | `False`         | 处理视频时，将单个帧保存为图片。适用于提取特定帧或进行详细的逐帧分析。                                                                        |
| `save_txt`    | `bool`         | `False`         | 将检测结果保存在文本文件中，格式为 `[class] [x_center] [y_center] [width] [height] [confidence]`。适用于与其他分析工具集成。                                       |
| `save_conf`   | `bool`         | `False`         | 在保存的文本文件中包含置信度分数。增强了后处理和分析可用的细节。                                                                          |
| `save_crop`   | `bool`         | `False`         | 保存检测区域的裁剪图片。适用于数据集增强、分析或创建针对特定对象的专注数据集。                                                                 |
| `show_labels` | `bool`         | `True`          | 在视觉输出中显示每个检测的标签。立即了解检测到的对象。                                                                                   |
| `show_conf`   | `bool`         | `True`          | 在标签旁边显示每个检测的置信度分数。提供模型对每个检测的信心见解。                                                                         |
| `show_boxes`  | `bool`         | `True`          | 绘制检测对象的边界框。对于图像或视频帧中对象的视觉识别和定位至关重要。                                                                       |
| `line_width`  | `None` 或 `int` | `None`          | 指定边界框的线宽。如果为 `None`，则根据图像大小自动调整线宽。提供视觉自定义以提高清晰度。                                                          |


## 图像和视频格式

YOLOv11支持多种图像和视频格式，具体请参见[ultralytics/data/utils.py](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/data/utils.py)。下表列出了有效的后缀和示例预测命令。

### 图像

下表包含有效的Ultralytics图像格式。

!!! 注意

    HEIC图像仅支持推理，不支持训练。

| 图像后缀 | 示例预测命令                     | 参考                                                                     |
| -------- | ------------------------------ | ------------------------------------------------------------------------- |
| `.bmp`   | `yolo predict source=image.bmp` | [微软BMP文件格式](https://en.wikipedia.org/wiki/BMP_file_format)               |
| `.dng`   | `yolo predict source=image.dng` | [Adobe DNG](https://en.wikipedia.org/wiki/Digital_Negative)                |
| `.jpeg`  | `yolo predict source=image.jpeg`| [JPEG](https://en.wikipedia.org/wiki/JPEG)                                    |
| `.jpg`   | `yolo predict source=image.jpg` | [JPEG](https://en.wikipedia.org/wiki/JPEG)                                    |
| `.mpo`   | `yolo predict source=image.mpo` | [多重图片对象](https://fileinfo.com/extension/mpo)                         |
| `.png`   | `yolo predict source=image.png`| [便携网络图形](https://en.wikipedia.org/wiki/PNG)                           |
| `.tif`   | `yolo predict source=image.tif` | [标签图像文件格式](https://en.wikipedia.org/wiki/TIFF)                       |
| `.tiff`  | `yolo predict source=image.tiff`| [标签图像文件格式](https://en.wikipedia.org/wiki/TIFF)                       |
| `.webp`  | `yolo predict source=image.webp`| [WebP](https://en.wikipedia.org/wiki/WebP)                                   |
| `.pfm`   | `yolo predict source=image.pfm` | [便携浮点映射](https://en.wikipedia.org/wiki/Netpbm#File_formats)            |
| `.HEIC`  | `yolo predict source=image.HEIC`| [高效图像格式](https://en.wikipedia.org/wiki/HEIF)                            |

### 视频文件格式

下表列出了Ultralytics支持的有效视频格式。

| 视频文件后缀 | 示例预测命令                  | 参考链接                                                                        |
| ------------- | ---------------------------- | ------------------------------------------------------------------------------- |
| `.asf`        | `yolo predict source=video.asf` | [高级系统格式](https://en.wikipedia.org/wiki/Advanced_Systems_Format)             |
| `.avi`        | `yolo predict source=video.avi` | [音频视频交错](https://en.wikipedia.org/wiki/Audio_Video_Interleave)               |
| `.gif`        | `yolo predict source=video.gif` | [图形交换格式](https://en.wikipedia.org/wiki/GIF)                                 |
| `.m4v`        | `yolo predict source=video.m4v` | [MPEG-4 Part 14](https://en.wikipedia.org/wiki/M4V)                              |
| `.mkv`        | `yolo predict source=video.mkv` | [Matroska](https://en.wikipedia.org/wiki/Matroska)                               |
| `.mov`        | `yolo predict source=video.mov` | [QuickTime文件格式](https://en.wikipedia.org/wiki/QuickTime_File_Format)           |
| `.mp4`        | `yolo predict source=video.mp4` | [MPEG-4 Part 14 - Wikipedia](https://en.wikipedia.org/wiki/MPEG-4_Part_14)       |
| `.mpeg`       | `yolo predict source=video.mpeg`| [MPEG-1 Part 2](https://en.wikipedia.org/wiki/MPEG-1)                            |
| `.mpg`        | `yolo predict source=video.mpg` | [MPEG-1 Part 2](https://en.wikipedia.org/wiki/MPEG-1)                            |
| `.ts`         | `yolo predict source=video.ts`  | [MPEG传输流](https://en.wikipedia.org/wiki/MPEG_transport_stream)               |
| `.wmv`        | `yolo predict source=video.wmv` | [Windows Media Video](https://en.wikipedia.org/wiki/Windows_Media_Video)          |
| `.webm`       | `yolo predict source=video.webm`| [WebM项目](https://en.wikipedia.org/wiki/WebM)                                   |

## 处理结果

所有 Ultralytics 的 `predict()` 调用都会返回 `Results` 对象的列表：

```python
from ultralytics import YOLO

# 加载预训练的YOLOv11n模型
model = YOLO("yolo11n.pt")

# 在一张图片上运行推断
results = model("bus.jpg")  # 包含1个Results对象的列表
results = model(["bus.jpg", "zidane.jpg"])  # 包含2个Results对象的列表
```

`Results` 对象具有以下属性：

| 属性         | 类型                  | 描述                                                                              |
| ------------ | --------------------- | ---------------------------------------------------------------------------------- |
| `orig_img`   | `numpy.ndarray`       | 原始图像，以 numpy 数组形式表示。                                                     |
| `orig_shape` | `tuple`               | 原始图像的形状，格式为 (高度, 宽度)。                                      |
| `boxes`      | `Boxes, optional`     | 包含检测边界框的 Boxes 对象。                                  |
| `masks`      | `Masks, optional`     | 包含检测掩码的 Masks 对象。                                           |
| `probs`      | `Probs, optional`     | 包含每个类别分类任务概率的 Probs 对象。           |
| `keypoints`  | `Keypoints, optional` | 包含每个对象检测关键点的 Keypoints 对象。                        |
| `obb`        | `OBB, optional`       | 包含定向边界框的 OBB 对象。                                        |
| `speed`      | `dict`                | 一个字典，包含每张图像的预处理、推理和后处理速度（以毫秒为单位）。 |
| `names`      | `dict`                | 一个包含类别名称的字典。                                                             |
| `path`       | `str`                 | 图像文件的路径。                                                              |

`Results` 对象具有以下方法：

| 方法        | 返回类型     | 描述                                                                         |
| ------------- | --------------- | ----------------------------------------------------------------------------------- |
| `update()`    | `None`          | 更新 Results 对象的 boxes、masks 和 probs 属性。                |
| `cpu()`       | `Results`       | 返回一个所有张量都在 CPU 内存中的 Results 对象副本。                 |
| `numpy()`     | `Results`       | 返回一个所有张量都是 numpy 数组的 Results 对象副本。               |
| `cuda()`      | `Results`       | 返回一个所有张量都在 GPU 内存中的 Results 对象副本。                 |
| `to()`        | `Results`       | 返回一个在指定设备和数据类型上的所有张量都在 Results 对象副本。 |
| `new()`       | `Results`       | 返回一个具有相同图像、路径和名称的新 Results 对象。                   |
| `plot()`      | `numpy.ndarray` | 绘制检测结果。返回一个标注图像的 numpy 数组。          |
| `show()`      | `None`          | 将标注结果展示到屏幕上。                                                   |
| `save()`      | `None`          | 将标注结果保存到文件中。                                                     |
| `verbose()`   | `str`           | 返回每个任务的日志字符串。                                                    |
| `save_txt()`  | `None`          | 将预测结果保存到一个 txt 文件中。                                                   |
| `save_crop()` | `None`          | 将裁剪后的预测结果保存到 `save_dir/cls/file_name.jpg`。                           |
| `tojson()`    | `str`           | 将对象转换为 JSON 格式。                                                  |

### 框

`Boxes` 对象可用于索引、操作以及将边界框转换为不同的格式。

```python
from ultralytics import YOLO

# 加载预训练的YOLOv11n模型
model = YOLO("yolo11n.pt")

# 在图像上运行推理
results = model("bus.jpg")  # 结果列表

# 查看结果
for r in results:
    print(r.boxes)  # 打印包含检测边界框的Boxes对象
```

以下是`Boxes`类方法和属性的表格，包括它们的名称、类型和描述：

| 名称     | 类型            | 描述                                                         |
| -------- | ------------ | ------------------------------------------------------------ |
| `cpu()`  | 方法          | 将对象移动到CPU内存。                                        |
| `numpy()`| 方法          | 将对象转换为numpy数组。                                      |
| `cuda()` | 方法          | 将对象移动到CUDA内存。                                      |
| `to()`   | 方法          | 将对象移动到指定的设备。                                     |
| `xyxy`   | 属性 (`torch.Tensor`) | 以xyxy格式返回框。                                          |
| `conf`   | 属性 (`torch.Tensor`) | 返回框的置信度值。                                          |
| `cls`    | 属性 (`torch.Tensor`) | 返回框的类别值。                                            |
| `id`     | 属性 (`torch.Tensor`) | 返回框的跟踪ID（如果有）。                                    |
| `xywh`   | 属性 (`torch.Tensor`) | 以xywh格式返回框。                                          |
| `xyxyn`  | 属性 (`torch.Tensor`) | 返回原始图像尺寸标准化后的xyxy格式框。                          |
| `xywhn`  | 属性 (`torch.Tensor`) | 返回原始图像尺寸标准化后的xywh格式框。                         |

### 掩码

`Masks` 对象可用于索引、操作和将掩码转换为分割。

```python
from ultralytics import YOLO

# 加载预训练的YOLO11n-seg分割模型
model = YOLO("yolo11n-seg.pt")

# 对一张图片进行推理
results = model("bus.jpg")  # 结果列表

# 查看结果
for r in results:
    print(r.masks)  # 打印包含检测到的实例遮罩的Masks对象
```

以下是 `Masks` 类的方法与属性的表格，包括它们的名称、类型和描述：

| 名称    | 类型        | 描述                                                         |
| ------- | ----------- | ------------------------------------------------------------ |
| `cpu()` | 方法        | 返回位于 CPU 内存中的掩码张量。                                 |
| `numpy()`| 方法        | 将掩码张量转换为 numpy 数组。                                   |
| `cuda()`| 方法        | 返回位于 GPU 内存中的掩码张量。                                 |
| `to()`  | 方法        | 返回具有指定设备和数据类型的掩码张量。                           |
| `xyn`   | 属性 (`torch.Tensor`) | 表示为张量的归一化段列表。                                    |
| `xy`    | 属性 (`torch.Tensor`) | 表示为张量的像素坐标段列表。                                  |

### 关键点

`Keypoints` 对象可用于索引、操作和规范化坐标。


```python
from ultralytics import YOLO

# Load a pretrained YOLO11n-pose Pose model
model = YOLO("yolo11n-pose.pt")

# Run inference on an image
results = model("bus.jpg")  # results list

# View results
for r in results:
    print(r.keypoints)  # print the Keypoints object containing the detected keypoints
```


以下是 `Keypoints` 类方法和属性的表格，包括它们的名称、类型和描述：

| 名称    | 类型         | 描述                                                       |
| ------- | ------------ | --------------------------------------------------------- |
| `cpu()` | 方法         | 返回在 CPU 内存中的关键点张量。                              |
| `numpy()` | 方法       | 返回作为 numpy 数组的关键点张量。                           |
| `cuda()` | 方法       | 返回在 GPU 内存中的关键点张量。                              |
| `to()`   | 方法         | 返回具有指定设备和数据类型的关键点张量。                   |
| `xyn`    | 属性 (`torch.Tensor`) | 一个表示为张量的归一化关键点列表。                       |
| `xy`     | 属性 (`torch.Tensor`) | 一个表示为张量的像素坐标中的关键点列表。                 |
| `conf`   | 属性 (`torch.Tensor`) | 如果可用，则返回关键点的置信度值，否则为 None。             |

### Probs

`Probs` 对象可用于索引，获取分类的 `top1` 和 `top5` 索引及分数。

```python
from ultralytics import YOLO

# 加载一个预训练的 YOLOv11n-cls 分类模型
model = YOLO("yolo11n-cls.pt")

# 在一张图片上运行推理
results = model("bus.jpg")  # 结果列表

# 查看结果
for r in results:
    print(r.probs)  # 打印包含检测到的类别概率的 Probs 对象
```

以下是对 `Probs` 类方法和属性的总结表：

| 名称       | 类型                     | 描述                                                              |
| --------- | ----------------------- | ----------------------------------------------------------------- |
| `cpu()`    | 方法                    | 返回一个位于CPU内存中的probs张量的副本。                             |
| `numpy()`  | 方法                    | 将probs张量转换为NumPy数组并返回其副本。                          |
| `cuda()`   | 方法                    | 返回一个位于GPU内存中的probs张量的副本。                             |
| `to()`     | 方法                    | 返回指定设备和数据类型下的probs张量的副本。                         |
| `top1`     | 属性（`int`）            | 最可能类别的索引。                                                   |
| `top5`     | 属性（`list[int]`）      | 前五个最可能类别的索引列表。                                         |
| `top1conf` | 属性（`torch.Tensor`）   | 最可能类别的置信度。                                                 |
| `top5conf` | 属性（`torch.Tensor`）   | 前五个最可能类别的置信度。                                           |

### OBB

OBB对象可用于索引、操作和转换方向性边界框到不同格式。

```python
from ultralytics import YOLO

# 加载预训练的YOLO11n模型
model = YOLO("yolo11n-obb.pt")

# 对图像进行推理
results = model("boats.jpg")  # 结果列表

# 查看结果
for r in results:
    print(r.obb)  # 打印包含方向性检测边界框的OBB对象
```

以下是`OBB`类方法和属性的表格，包括它们的名称、类型和描述：

| 名称       | 类型                      | 描述                                                           |
| --------- | ------------------------- | ------------------------------------------------------------ |
| `cpu()`   | 方法                      | 将对象移动到CPU内存。                                        |
| `numpy()` | 方法                      | 将对象转换为NumPy数组。                                      |
| `cuda()`  | 方法                      | 将对象移动到CUDA内存。                                       |
| `to()`    | 方法                      | 将对象移动到指定设备。                                        |
| `conf`    | 属性（`torch.Tensor`）   | 返回框的置信度值。                                          |
| `cls`     | 属性（`torch.Tensor`）   | 返回框的类别值。                                             |
| `id`      | 属性（`torch.Tensor`）   | 返回框的跟踪ID（如果可用）。                                   |
| `xyxy`    | 属性（`torch.Tensor`）   | 返回xyxy格式的水平框。                                        |
| `xywhr`   | 属性（`torch.Tensor`）   | 返回xywhr格式的旋转框。                                       |
| `xyxyxyxy`| 属性（`torch.Tensor`）   | 返回xyxyxyxy格式的旋转框。                                     |
| `xyxyxyxyn`| 属性（`torch.Tensor`） | 返回按图像大小归一化的xyxyxyxy格式的旋转框。               |

## 绘制结果

在`Results`对象中，`plot()`方法通过将检测到的对象（例如边界框、遮罩、关键点和概率）覆盖到原始图像上，便于可视化预测。此方法返回一个作为NumPy数组的注释图像，方便显示或保存。


```python
from PIL import Image

from ultralytics import YOLO

# 加载预训练的YOLO11n模型
model = YOLO("yolo11n.pt")

# 对'bus.jpg'进行推理
results = model(["bus.jpg", "zidane.jpg"])  # 结果列表

# 可视化结果
for i, r in enumerate(results):
    # 绘制结果图像
    im_bgr = r.plot()  # BGR顺序的numpy数组
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB顺序的PIL图像

    # 在支持的环境中显示结果
    r.show()

    # 将结果保存到磁盘
    r.save(filename=f"results{i}.jpg")
```

### `plot()` 方法参数

`plot()` 方法支持各种参数以定制输出：

| 参数         | 类型            | 描述                                                                | 默认值      |
| ------------ | --------------- | -------------------------------------------------------------------------- | ----------- |
| `conf`       | `bool`          | 是否包含检测置信度分数。                                             | `True`      |
| `line_width` | `float`         | 边界框的线宽。如果为 `None`，则根据图像大小缩放。                 | `None`      |
| `font_size`  | `float`         | 文本字体大小。如果为 `None`，则根据图像大小缩放。                    | `None`      |
| `font`       | `str`           | 文本注释的字体名称。                                                | `'Arial.ttf'` |
| `pil`        | `bool`          | 返回图像作为 PIL Image 对象。                                        | `False`     |
| `img`        | `numpy.ndarray` | 绘图使用的替代图像。如果为 `None`，则使用原始图像。                    | `None`      |
| `im_gpu`     | `torch.Tensor`  | 用于更快蒙版绘制的 GPU 加速图像。形状为 (1, 3, 640, 640)。             | `None`      |
| `kpt_radius` | `int`           | 绘制关键点的半径。                                                  | `5`         |
| `kpt_line`   | `bool`          | 用线连接关键点。                                                    | `True`      |
| `labels`     | `bool`          | 在注释中包含类别标签。                                              | `True`      |
| `boxes`      | `bool`          | 在图像上叠加边界框。                                                | `True`      |
| `masks`      | `bool`          | 在图像上叠加掩码。                                                  | `True`      |
| `probs`      | `bool`          | 包含分类概率。                                                      | `True`      |
| `show`       | `bool`          | 使用默认图像查看器直接显示注释后的图像。                               | `False`     |
| `save`       | `bool`          | 将注释后的图像保存到通过 `filename` 指定的文件中。                  | `False`     |
| `filename`   | `str`           | 如果 `save` 为真，保存注释后图像的文件路径和名称。                    | `None`      |
| `color_mode` | `str`           | 指定颜色模式，例如 'instance' 或 'class'。                            | `'class'`    |

## 线程安全推理

当您在不同的线程中并行运行多个YOLO模型时，确保推理过程中的线程安全至关重要。线程安全的推理保证每个线程的预测是相互隔离的，不会互相干扰，避免竞态条件，并确保输出的一致性和可靠性。

在使用多线程应用程序中的YOLO模型时，为每个线程实例化单独的模型对象或使用线程本地存储以防止冲突非常重要：

在每个线程内部实例化单个模型以进行线程安全的推理：
```python
from threading import Thread

from ultralytics import YOLO

def thread_safe_predict(model, image_path):
    """使用本地实例化的YOLO模型对图像进行线程安全的预测。"""
    model = YOLO(model)
    results = model.predict(image_path)
    # 处理结果

# 启动每个线程，每个线程都有自己的模型实例
Thread(target=thread_safe_predict, args=("yolo11n.pt", "image1.jpg")).start()
Thread(target=thread_safe_predict, args=("yolo11n.pt", "image2.jpg")).start()
```

有关YOLO模型线程安全推理的深入探讨和逐步指导，请参阅[YOLO线程安全推理指南](https://docs.ultralytics.com/zh/guides/yolo-thread-safe-inference/#how-can-i-avoid-race-conditions-when-using-yolo-models-in-a-multi-threaded-python-environment)。该指南将提供所有必要的信息，帮助您避免常见错误并确保您的多线程推理顺利进行。

## 流媒体源 `for` 循环示例

这里有使用 OpenCV (`cv2`) 和 YOLO 在视频帧上运行推理的 Python 脚本。此脚本假设您已经安装了必要的包（`opencv-python` 和 `ultralytics`）。


```python
import cv2
from ultralytics import YOLO

# 加载 YOLO 模型
model = YOLO("yolo11n.pt")

# 打开视频文件
video_path = "path/to/your/video/file.mp4"
cap = cv2.VideoCapture(video_path)

# 遍历视频帧
while cap.isOpened():
    # 从视频中读取一帧
    success, frame = cap.read()

    if success:
        # 在帧上运行 YOLO 推理
        results = model(frame)

        # 在帧上可视化结果
        annotated_frame = results[0].plot()

        # 显示带有注释的帧
        cv2.imshow("YOLO 推理", annotated_frame)

        # 如果按下 'q'，退出循环
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # 如果到达视频末尾，退出循环
        break

# 释放视频捕获对象并关闭显示窗口
cap.release()
cv2.destroyAllWindows()
```

该脚本将对视频的每一帧进行预测，将结果显示出来，并在窗口中显示。通过按 'q' 可以退出循环。

## 真实世界应用

**制造** | **体育** | **安全**
| :-----------------------------------------------: | :--------------------------------------------------: | :-----------------------------------------: |
| ![汽车零件检测][car spare parts] | ![足球运动员检测][football player detect] | ![跌倒人员检测][human fall detect] |
|           汽车零件检测           |              足球运动员检测               |           跌倒人员检测            |

[car spare parts]: https://github.com/RizwanMunawar/ultralytics/assets/62513924/a0f802a8-0776-44cf-8f17-93974a4a28a1
[football player detect]: https://github.com/RizwanMunawar/ultralytics/assets/62513924/7d320e1f-fc57-4d7f-a691-78ee579c3442
[human fall detect]: https://github.com/RizwanMunawar/ultralytics/assets/62513924/86437c4a-3227-4eee-90ef-9efb697bdb43
